[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/monacofj/moeabench/blob/main/examples/example_15.ipynb)

# Example 15: The Triple-Mode Hypervolume (Raw, Relative, Absolute)

This example demonstrates the three scaling perspectives available for Hypervolume in MoeaBench:
1. **'raw'**: Physical volume conquered ($H_{raw}$). Invariant to neighbors or competitors.
2. **'relative'**: Competitive efficiency ($H_{rel}$). Scaled by the best observed session front, forcing a 1.0 ceiling for the current winner.
3. **'absolute'**: Theoretical optimality ($H_{abs}$). Scaled by the mathematical **Ground Truth** (requires calibration).

> **Terminology Note**: In this example, we compare a **Baseline** configuration (20 individuals) against a **Premium** configuration (100 individuals) to show how they behave under different normalization schemes.

> **Visual Note**: The mathematical shape of the curves across these three modes is identical because moving between scales merely divides the physical raw volume by a distinct scalar constant. What changes is the vertical scaling and where the curves land relative to a 1.0 ceiling constraint.

In [ ]:
# Install MoeaBench from GitHub (Uncomment if running in Colab)
# !pip install --quiet git+https://github.com/monacofj/moeabench.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from MoeaBench import mb
print(f"MoeaBench v{mb.system.version()}")

## 1. Setup the Problem and Calibration

We use **DTLZ2** (3 objectives). To enable the **Absolute** mode, we must first ensure the problem is calibrated.

In [ ]:
dtlz2 = mb.mops.DTLZ2(n_var=7, n_obj=3)
dtlz2.calibrate()

print("\nRunning Experiment 1 (Baseline: 20 individuals)...\n")
exp1 = mb.experiment(dtlz2, mb.moeas.NSGA2(population=20, generations=50))
exp1.name = "Baseline NSGA-II"
exp1.run(repeat=5)

print("\nRunning Experiment 2 (Premium: 100 individuals)...\n")
exp2 = mb.experiment(dtlz2, mb.moeas.NSGA2(population=100, generations=50))
exp2.name = "Premium NSGA-II"
exp2.run(repeat=5)

## 2. Calculating the Three Perspectives

We calculate the hypervolume for both experiments using the three scales.

In [ ]:
global_ref = [exp1, exp2]

# A) Raw: Physical volume ($H_{raw}$)
hv1_raw = mb.metrics.hv(exp1, ref=global_ref, scale='raw')
hv2_raw = mb.metrics.hv(exp2, ref=global_ref, scale='raw')

# B) Relative: Competitive Efficiency ($H_{rel}$)
hv1_rel = mb.metrics.hv(exp1, ref=global_ref, scale='relative')
hv2_rel = mb.metrics.hv(exp2, ref=global_ref, scale='relative')

# C) Absolute: Theoretical Optimality ($H_{abs}$)
hv1_abs = mb.metrics.hv(exp1, scale='absolute')
hv2_abs = mb.metrics.hv(exp2, scale='absolute')

print("--- RESULTS AT FINAL GENERATION ---")
print(f"-> Raw:  exp1={hv1_raw.values[-1,:].mean():.4f}, exp2={hv2_raw.values[-1,:].mean():.4f}")
print(f"-> Rel:  exp1={hv1_rel.values[-1,:].mean():.4f}, exp2={hv2_rel.values[-1,:].mean():.4f}")
print(f"-> Abs:  exp1={hv1_abs.values[-1,:].mean():.4f}, exp2={hv2_abs.values[-1,:].mean():.4f}")

## 3. Visual Comparison

We plot the three modes side-by-side using unified axes and annotated reference lines.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

v1_raw, v2_raw = hv1_raw.values[-1,:].mean(), hv2_raw.values[-1,:].mean()
v1_rel, v2_rel = hv1_rel.values[-1,:].mean(), hv2_rel.values[-1,:].mean()
v1_abs, v2_abs = hv1_abs.values[-1,:].mean(), hv2_abs.values[-1,:].mean()

# Unified Y-axis based on Raw
y_max = max(1.05, v2_raw * 1.05)

def annotate_lines(ax, val1, val2):
    ax.axhline(val1, color='blue', linestyle=':', alpha=0.5)
    ax.text(0.5, val1 - y_max*0.01, f"hv1 = {val1:.3f}", color='blue', fontsize=8, va='top', ha='left')
    ax.axhline(val2, color='darkorange', linestyle=':', alpha=0.5)
    ax.text(0.5, val2 - y_max*0.01, f"hv2 = {val2:.3f}", color='darkorange', fontsize=8, va='top', ha='left')
    ax.set_ylim([0, y_max])

# Plotting without manual labels to verify engine standardization
mb.view.perf_history(hv1_raw, hv2_raw, ax=ax1, title="1. Physical (Raw)")
annotate_lines(ax1, v1_raw, v2_raw)

mb.view.perf_history(hv1_rel, hv2_rel, ax=ax2, title="2. Competitive (Relative)")
annotate_lines(ax2, v1_rel, v2_rel)
ax2.axhline(1.0, color='gray', linestyle='--', alpha=0.7, label="Session Winner (1.0)")
ax2.legend(loc='lower right', fontsize=8)

mb.view.perf_history(hv1_abs, hv2_abs, ax=ax3, title="3. Theoretical (Absolute)")
annotate_lines(ax3, v1_abs, v2_abs)
ax3.axhline(1.0, color='gold', linestyle='--', alpha=1.0, label="Ground Truth (1.0)")
ax3.legend(loc='lower right', fontsize=8)

plt.tight_layout()
plt.show()

## Conclusion

1. **RAW** is an invariant physical scale independent of competitors.
2. **RELATIVE** ($H_{rel}$) measures efficiency against the session winner.
3. **ABSOLUTE** ($H_{abs}$) measures optimality against the Ground Truth.